In [1]:
# import required modules
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
# Setting the options to display maximum rows and columns in the output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

##### Step 1 - load the dataset and understand it

In [3]:
# load the excel file and store it in a variable called df
df = pd.read_excel(r"C:\Users\aswin\OneDrive\Documents\Crime_Data.xlsx")

In [4]:
df.shape

(549999, 22)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549999 entries, 0 to 549998
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ID                    549999 non-null  int64         
 1   Case Number           549999 non-null  object        
 2   Date                  549999 non-null  datetime64[ns]
 3   Block                 549999 non-null  object        
 4   IUCR                  549999 non-null  object        
 5   Primary Type          549999 non-null  object        
 6   Description           549999 non-null  object        
 7   Location Description  548326 non-null  object        
 8   Arrest                549999 non-null  bool          
 9   Domestic              549999 non-null  bool          
 10  Beat                  549999 non-null  int64         
 11  District              549999 non-null  int64         
 12  Ward                  213674 non-null  float64       
 13 

In [6]:
df.head(3)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,5741943,HN549294,2007-08-25 09:22:18,074XX N ROGERS AVE,560,ASSAULT,SIMPLE,OTHER,False,False,2422,24,49.0,1.0,08A,NaN,NaN,2007,2015-08-17 15:03:40,NaN,NaN,NaN
1,25953,JE240540,2021-05-24 15:06:00,020XX N LARAMIE AVE,110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,2515,25,36.0,19.0,01A,1141387.0,1913179.0,2021,2023-11-18 15:39:49,41.917838,-87.755969,"(41.917838056, -87.755968972)"
2,26038,JE279849,2021-06-26 09:24:00,062XX N MC CORMICK RD,110,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,True,False,1711,17,50.0,13.0,01A,1152781.0,1941458.0,2021,2023-11-18 15:39:49,41.995219,-87.713355,"(41.995219444, -87.713354912)"


##### Step 2 -- Find misspelled words in the dataset and replace it with meaningful word

In [7]:
# category columns in the dataset
cat_cols = df.select_dtypes(include='object').columns
cat_cols

Index(['Case Number', 'Block', 'IUCR', 'Primary Type', 'Description',
       'Location Description', 'FBI Code', 'Location'],
      dtype='object')

In [8]:
# Unique values in all category columns
for i in cat_cols:
    print(f'unique values in {i} column')
    print(df[i].unique())
    print('-'*70)

unique values in Case Number column
['HN549294' 'JE240540' 'JE279849' ... 'G583569' 'G610965' 'G627247']
----------------------------------------------------------------------
unique values in Block column
['074XX N ROGERS AVE' '020XX N LARAMIE AVE' '062XX N MC CORMICK RD' ...
 '042XX W DRUMMOND PL' '030XX E 106 ST' '040XX W 70 ST']
----------------------------------------------------------------------
unique values in IUCR column
[560 110 620 454 820 1320 1153 281 1310 810 1120 890 1110 1790 910 1156
 '031A' 266 325 1753 1565 2826 1752 1544 1562 1477 1751 486 1754 583 265
 860 1350 1220 1330 920 497 5002 '502P' 520 '502R' '041A' 320 1513 1152
 420 1020 1340 460 2820 630 610 4387 '051A' 2027 1365 930 498 1710 580
 4230 330 '031B' 313 1563 1130 530 1360 1478 1780 '143A' 1121 430 1345
 1210 2024 1150 3710 326 1581 1206 2850 1195 479 2890 917 1750 2029 '033A'
 484 870 '141B' 462 2021 5000 331 650 1151 5011 1200 2825 1812 1154 927
 2092 1242 2022 2028 584 1155 '141C' 1822 495 1140 263 1582

In [9]:
# unique_values = df['Block'].unique()
# for value in unique_values:
#     matches = process.extract(value, unique_values, limit=5, scorer=fuzz.partial_ratio)
#     print(f"{value}: {matches}")

In [10]:
grouped_descriptions = df.groupby('IUCR')['Description'].unique()
descriptions = grouped_descriptions[grouped_descriptions.apply(len) > 1]
pd.set_option('display.max_colwidth', None)
print(descriptions)

IUCR
261                                                                                   [AGGRAVATED - HANDGUN, AGGRAVATED: HANDGUN]
262                                                                       [AGGRAVATED - OTHER FIREARM, AGGRAVATED: OTHER FIREARM]
263                                                        [AGGRAVATED - KNIFE / CUTTING INSTRUMENT, AGGRAVATED: KNIFE/CUT INSTR]
264                                                          [AGGRAVATED - OTHER DANGEROUS WEAPON, AGGRAVATED: OTHER DANG WEAPON]
265                                                                                       [AGGRAVATED - OTHER, AGGRAVATED: OTHER]
271                                                                          [ATTEMPT AGGRAVATED - HANDGUN, ATTEMPT AGG: HANDGUN]
273                                               [ATTEMPT AGGRAVATED - KNIFE / CUTTING INSTRUMENT, ATTEMPT AGG: KNIFE/CUT INSTR]
274                                                 [ATTEMPT AGGRAVATED - OTHER DANGE

In [11]:
# df['Description'] = df['Description'].replace({
#     'AGGRAVATED: HANDGUN' : 'AGGRAVATED - HANDGUN',
#     'AGGRAVATED: OTHER FIREARM' : 'AGGRAVATED - OTHER FIREARM',
#     'AGGRAVATED: KNIFE/CUT INSTR' : 'AGGRAVATED - KNIFE / CUTTING INSTRUMENT',
#     'AGGRAVATED: OTHER DANG WEAPON' : 'AGGRAVATED - OTHER DANGEROUS WEAPON',
#     'AGGRAVATED - OTHER DANGEROUS WEAPON' : 'AGGRAVATED - OTHER',
#     'ATTEMPT AGG: HANDGUN' : 'ATTEMPT AGGRAVATED - HANDGUN',
#     'ATTEMPT AGG: KNIFE/CUT INSTR' : 'ATTEMPT AGGRAVATED - KNIFE / CUTTING INSTRUMENT',
#     'ATTEMPT AGG: OTHER DANG WEAPON' : 'ATTEMPT AGGRAVATED - OTHER DANGEROUS WEAPON',
#     'ATTEMPT AGG: OTHER' : 'ATTEMPT AGGRAVATED - OTHER',
#     'ARMED:KNIFE/CUTTING INSTRUMENT' : 'ARMED - KNIFE / CUTTING INSTRUMENT',
#     'ARMED: OTHER DANGEROUS WEAPON' : 'ARMED - OTHER DANGEROUS WEAPON',
#     'STRONGARM - NO WEAPON' : 'STRONG ARM - NO WEAPON',
#     'ATTEMPT: AGGRAVATED' : 'ATTEMPT AGGRAVATED',
#     'ATTEMPT: ARMED-KNIFE/CUT INSTR' : 'ATTEMPT ARMED - KNIFE / CUTTING INSTRUMENT',
#     'ATTEMPT: ARMED-OTHER DANG WEAP' : 'ATTEMPT ARMED - OTHER DANGEROUS WEAPON',
#     'ATTEMPT: STRONGARM-NO WEAPON' : 'ATTEMPT STRONG ARM - NO WEAPON',
#     'AGGRAVATED:KNIFE/CUTTING INSTR' : 'AGGRAVATED - KNIFE / CUTTING INSTRUMENT',
#     'AGGRAVATED: OTHER DANG WEAPON' : 'AGGRAVATED - OTHER DANGEROUS WEAPON',
#     'AGG: HANDS/FIST/FEET NO/MINOR INJURY' : 'AGGRAVATED - HANDS, FISTS, FEET, NO / MINOR INJURY',
#     'AGGRAVATED PO: HANDGUN' : 'AGGRAVATED POLICE OFFICER - HANDGUN',
#     'AGGRAVATED PO: KNIFE/CUT INSTR' : 'AGGRAVATED POLICE OFFICER - KNIFE / CUTTING INSTRUMENT',
#     'AGGRAVATED PO: OTHER DANG WEAP' : 'AGGRAVATED POLICE OFFICER - OTHER DANGEROUS WEAPON',
#     'AGG PO HANDS NO/MIN INJURY' : 'AGGRAVATED PO - HANDS, FISTS, FEET, NO / MINOR INJURY',
#     'AGG PRO.EMP:KNIFE/CUTTING INST' : 'AGGRAVATED PROTECTED EMPLOYEE - KNIFE / CUTTING INSTRUMENT',
#     'AGG PRO.EMP: OTHER DANG WEAPON' : 'AGGRAVATED PROTECTED EMPLOYEE - OTHER DANGEROUS WEAPON',
#     'PRO EMP HANDS NO/MIN INJURY' : 'PROTECTED EMPLOYEE - HANDS, FISTS, FEET, NO / MINOR INJURY',
#     'AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST' : 'AGGRAVATED DOMESTIC BATTERY - KNIFE / CUTTING INSTRUMENT',
#     'AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON' : 'AGGRAVATED DOMESTIC BATTERY - OTHER DANGEROUS WEAPON',
#     'AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY' : 'AGGRAVATED DOMESTIC BATTERY - HANDS, FISTS, FEET, SERIOUS INJURY',
#     'AGG. DOMESTIC BATTERY - HANDS, FISTS, FEET, SERIOUS INJURY' : 'AGGRAVATED DOMESTIC BATTERY - HANDS, FISTS, FEET, SERIOUS INJURY',
#     'AGGRAVATED:KNIFE/CUTTING INSTR' : 'AGGRAVATED - KNIFE / CUTTING INSTRUMENT',
#     'AGGRAVATED: OTHER DANG WEAPON' : 'AGGRAVATED - OTHER DANGEROUS WEAPON',
#     'PRO EMP HANDS NO/MIN INJURY' : 'PROTECTED EMPLOYEE - HANDS, FISTS, FEET, NO / MINOR INJURY',
#     'AGGRAVATED PO: HANDGUN' : 'AGGRAVATED POLICE OFFICER - HANDGUN',
#     'AGGRAVATED PO: OTHER FIREARM' : 'AGGRAVATED POLICE OFFICER - OTHER FIREARM',
#     'AGGRAVATED PO:KNIFE/CUT INSTR' : 'AGGRAVATED POLICE OFFICER - KNIFE / CUTTING INSTRUMENT',
#     'AGGRAVATED PO: OTHER DANG WEAP' : 'AGGRAVATED POLICE OFFICER - OTHER DANGEROUS WEAPON',
#     'AGG PO HANDS NO/MIN INJURY' : 'AGGRAVATED POLICE OFFICER - HANDS, FISTS, FEET, NO INJURY',
#     'OVER $500' : 'THEFT AMOUNT OVER $300',
#     'OVER $300' : 'THEFT AMOUNT OVER $300',
#     '$500 AND UNDER' : 'THEFT AMOUNT UNDER $500',
#     '$300 AND UNDER' : 'THEFT AMOUNT UNDER $500',
#     'FINANCIAL ID THEFT: OVER $300' : 'FINANCIAL IDENTITY THEFT - OVER $300',
#     'FINANCIAL IDENTITY THEFT: OVER $300' : 'FINANCIAL IDENTITY THEFT - OVER $300',
#     'FINANCIAL ID THEFT:$300 &UNDER' : 'FINANCIAL IDENTITY THEFT - $300 & UNDER',
#     'FINANCIAL IDENTITY THEFT: $300 & UNDER' : 'FINANCIAL IDENTITY THEFT - $300 & UNDER',
#     'CYCLE, SCOOTER, BIKE W-VIN' : 'CYCLE, SCOOTER, BIKE WITH VIN',
#     'THEFT/RECOVERY: AUTOMOBILE' : 'THEFT / RECOVERY - AUTOMOBILE',
#     'THEFT/RECOVERY: TRUCK,BUS,MHOME' : 'THEFT / RECOVERY - TRUCK, BUS, MOBILE HOME',
#     'THEFT/RECOVERY: CYCLE, SCOOTER, BIKE W-VIN' : 'THEFT / RECOVERY - CYCLE, SCOOTER, BIKE WITH VIN',
#     'POS: EXPLOSIVE/INCENDIARY DEV' : 'POSSESSION - EXPLOSIVE / INCENDIARY DEVICE',
#     'FINAN EXPLOIT-ELDERLY/DISABLED' : 'FINANCIAL EXPLOITATION OF AN ELDERLY OR DISABLED PERSON',
#     'STOLEN PROP: BUY/RECEIVE/POS.' : 'STOLEN PROPERTY - BUY / RECEIVE / POSSESS',
#     'THEFT BY LESSEE,MOTOR VEH' : 'THEFT BY LESSEE, MOTOR VEHICLE',
#     'THEFT OF LABOR/SERVICES' : 'THEFT OF LABOR / SERVICES',
#     'THEFT OF LOST/MISLAID PROP' : 'THEFT OF LOST / MISLAID PROPERTY',
#     'TO STATE SUP PROP' : 'TO STATE SUPPORTED PROPERTY',
#     'DEFACE IDENT MARKS OF FIREARM' : 'DEFACE IDENTIFICATION MARKS OF FIREARM',
#     'POSS FIREARM/AMMO:NO FOID CARD' : 'POSSESS FIREARM / AMMUNITION - NO FOID CARD',
#     'SOLICIT FOR BUSINESS' : 'SOLICITING FOR BUSINESS',
#     'SALE/DIST OBSCENE MAT TO MINOR' : 'SALE / DISTRIBUTE OBSCENE MATERIAL TO MINOR',
#     'AGG CRIMINAL SEXUAL ABUSE' : 'AGGRAVATED CRIMINAL SEXUAL ABUSE',
#     'INDECENT SOLICITATION/CHILD' : 'INDECENT SOLICITATION OF A CHILD',
#     'INDECENT SOLICITATION/ADULT' : 'INDECENT SOLICITATION OF AN ADULT',
#     'SEX RELATION IN FAMILY' : 'SEXUAL RELATIONS IN FAMILY',
#     'NON-CONSENSUAL DISSEMINATION PRIVATE SEXUAL IMAGES' : 'NON-CONSENSUAL DISSEMINATION OF PRIVATE SEXUAL IMAGES',
#     'ATT AGG CRIMINAL SEXUAL ABUSE' : 'ATTEMPT AGGRAVATED CRIMINAL SEXUAL ABUSE',
#     'ENDANGERING LIFE/HEALTH CHILD' : 'ENDANGERING LIFE / HEALTH OF CHILD',
#     'CONTRIBUTE DELINQUENCY OF A CHILD' : 'CONTRIBUTE TO THE DELINQUENCY OF CHILD',
#     'CRIM SEX ABUSE BY FAM MEMBER' : 'CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER',
#     'AGG CRIM SEX ABUSE FAM MEMBER' : 'AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER',
#     'SEX ASSLT OF CHILD BY FAM MBR' : 'SEXUAL ASSAULT OF CHILD BY FAMILY MEMBER',
#     'AGG SEX ASSLT OF CHILD FAM MBR' : 'AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY MEMBER',
#     'CHILD ABDUCTION/STRANGER' : 'CHILD ABDUCTION / STRANGER',
#     'POSS: CANNABIS 30GMS OR LESS' : 'POSSESS - CANNABIS 30 GRAMS OR LESS',
#     'POSS: CANNABIS MORE THAN 30GMS' : 'POSSESS - CANNABIS MORE THAN 30 GRAMS',
#     'MANU/DEL:CANNABIS 10GM OR LESS' : 'MANUFACTURE / DELIVER - CANNABIS 10 GRAMS OR LESS',
#     'MANU/DEL:CANNABIS OVER 10 GMS' : 'MANUFACTURE / DELIVER - CANNABIS OVER 10 GRAMS',
#     'DELIVER CANNABIS TO PERSON <18' : 'DELIVER CANNABIS TO PERSON UNDER 18',
#     'MANU/DELIVER:AMPHETAMINES' : 'MANUFACTURE / DELIVER - AMPHETAMINES',
#     'MANU/DELIVER:BARBITUATES' : 'MANUFACTURE / DELIVER - BARBITURATES',
#     'MANU/DELIVER:COCAINE' : 'MANUFACTURE / DELIVER - COCAINE',
#     'MANU/DELIVER: HEROIN(BRN/TAN)' : 'MANUFACTURE / DELIVER - HEROIN (TAN / BROWN TAR)',
#     'MANU/DELIVER: HEROIN (WHITE)' : 'MANUFACTURE / DELIVER -  HEROIN (WHITE)',
#     'MANU/DELIVER: HALLUCINOGEN' : 'MANUFACTURE / DELIVER - HALLUCINOGEN',
#     'MANU/DELIVER:PCP' : 'MANUFACTURE / DELIVER - PCP',
#     'MANU/DELIVER:CRACK' : 'MANUFACTURE / DELIVER - CRACK',
#     'MANU/DELIVER:SYNTHETIC DRUGS' : 'MANUFACTURE / DELIVER - SYNTHETIC DRUGS',
#     'POSS: AMPHETAMINES' : 'POSSESS - AMPHETAMINES',
#     'POSS: BARBITUATES' : 'POSSESS - BARBITURATES',
#     'POSS: COCAINE' : 'POSSESS - COCAINE',
#     'POSS: HEROIN(BRN/TAN)' : 'POSSESS - HEROIN (TAN / BROWN TAR)',
#     'POSS: HEROIN(WHITE)' : 'POSSESS - HEROIN (WHITE)',
#     'POSS: HALLUCINOGENS' : 'POSSESS - HALLUCINOGENS',
#     'POSS: PCP' : 'POSSESS - PCP',
#     'POSS: CRACK' : 'POSSESS - CRACK',
#     'POSS: SYNTHETIC DRUGS' : 'POSSESS - SYNTHETIC DRUGS',
#     'POSS: HEROIN(BLACK TAR)' : 'POSSESS - HEROIN (BLACK TAR)',
#     'MANU/DELIVER:LOOK-ALIKE DRUG' : 'MANUFACTURE / DELIVER - LOOK-ALIKE DRUG',
#     'POSS: LOOK-ALIKE DRUGS' : 'POSSESS - LOOK-ALIKE DRUGS',
#     'ALTER/FORGE PRESCRIPTION' : 'ALTER / FORGE PRESCRIPTION',
#     'SOLICIT NARCOTICS ON PUBLICWAY' : 'SOLICIT NARCOTICS ON PUBLIC WAY',
#     'SELL/GIVE/DEL LIQUOR TO MINOR' : 'SELL / GIVE / DELIVER LIQUOR TO MINOR',
#     'UNLAWFUL USE/SALE AIR RIFLE' : 'UNLAWFUL USE / SALE OF AIR RIFLE',
#     'RESIST/OBSTRUCT/DISARM OFFICER' : '[RESIST / OBSTRUCT / DISARM OFFICER',
#     'UNLAWFUL INTERFERE/VISITATION' : 'UNLAWFUL VISITATION INTERFERENCE',
#     'VIO BAIL BOND: DOM VIOLENCE' : 'VIOLATION OF BAIL BOND - DOMESTIC VIOLENCE',
#     'SEX OFFENDER: FAIL TO REGISTER' : 'SEX OFFENDER - FAIL TO REGISTER',
#     'SEX OFFENDER: FAIL REG NEW ADD' : 'SEX OFFENDER - FAIL TO REGISTER NEW ADDRESS',
#     'SEX OFFENDER: PROHIBITED ZONE' : 'SEX OFFENDER - PROHIBITED ZONE',
#     'OTHER ARSON/EXPLOSIVE INCIDENT' : 'OTHER ARSON / EXPLOSIVE INCIDENT',
#     'ATT CRIM SEXUAL ABUSE' : 'ATTEMPT CRIMINAL SEXUAL ABUSE',
#     'ARMED: HANDGUN' : 'ARMED - HANDGUN',
#     'ARMED: OTHER FIREARM' : 'ARMED - OTHER FIREARM',
#     'ATTEMPT: ARMED-HANDGUN' : 'ATTEMPT ARMED - HANDGUN',
#     'ATTEMPT: ARMED-OTHER FIREARM' : 'ATTEMPT ARMED - OTHER FIREARM',
#     'AGGRAVATED: HANDGUN' : 'AGGRAVATED - HANDGUN',
#     'AGGRAVATED: OTHER FIREARM' : 'AGGRAVATED - OTHER FIREARM',
#     'AGGRAVATED: HANDGUN' : 'AGGRAVATED - HANDGUN',
#     'AGGRAVATED: OTHER FIREARM' : 'AGGRAVATED - OTHER FIREARM',
#     'UNLAWFUL USE HANDGUN' : 'UNLAWFUL USE - HANDGUN',
#     'UNLAWFUL USE OTHER FIREARM' : 'UNLAWFUL USE - OTHER FIREARM',
#     'UNLAWFUL USE OTHER DANG WEAPON' : 'UNLAWFUL USE - OTHER DANGEROUS WEAPON',
#     'UNLAWFUL SALE HANDGUN' : 'UNLAWFUL SALE - HANDGUN',
#     'UNLAWFUL POSS OF HANDGUN' : 'UNLAWFUL POSSESSION - HANDGUN',
#     'UNLAWFUL POSS OTHER FIREARM' : 'UNLAWFUL POSSESSION - OTHER FIREARM',
#     'UNLAWFUL POSS AMMUNITION' : 'UNLAWFUL POSSESSION - AMMUNITION',
#     'FALSE/STOLEN/ALTERED TRP' : 'FALSE / STOLEN / ALTERED TRP'
#     })

In [12]:
# Captialize the values in the column
def captialize_values(dataframe, column_name):
    try:
        dataframe[column_name] = dataframe[column_name].apply(lambda x: x.upper() if x!=x.upper() else x)
        print(f'successfully captialized the {column_name} column')
    except Exception as e:
        print(f'Error occured when captialize the values in {column_name} column :',e)

In [13]:
captialize_values(df, 'Block')  # captialize the values in the 'Block' column

# Handled the 'Primary Type' column where values have the same meaning but with variations and different spelling
df['Primary Type'] = df['Primary Type'].replace({'CRIM SEXUAL ASSAULT': 'CRIMINAL SEXUAL ASSAULT','OTHER NARCOTIC VIOLATION': 'NARCOTICS'})  

# Handled the 'Description' column where values have the same meaning but with variations and different spelling
df['Description'] = df['Description'].replace({
    'AGGRAVATED: HANDGUN' : 'AGGRAVATED - HANDGUN',
    'AGGRAVATED: OTHER FIREARM' : 'AGGRAVATED - OTHER FIREARM',
    'AGGRAVATED: KNIFE/CUT INSTR' : 'AGGRAVATED - KNIFE / CUTTING INSTRUMENT',
    'AGGRAVATED: OTHER DANG WEAPON' : 'AGGRAVATED - OTHER DANGEROUS WEAPON',
    'AGGRAVATED - OTHER DANGEROUS WEAPON' : 'AGGRAVATED - OTHER',
    'ATTEMPT AGG: HANDGUN' : 'ATTEMPT AGGRAVATED - HANDGUN',
    'ATTEMPT AGG: KNIFE/CUT INSTR' : 'ATTEMPT AGGRAVATED - KNIFE / CUTTING INSTRUMENT',
    'ATTEMPT AGG: OTHER DANG WEAPON' : 'ATTEMPT AGGRAVATED - OTHER DANGEROUS WEAPON',
    'ATTEMPT AGG: OTHER' : 'ATTEMPT AGGRAVATED - OTHER',
    'ARMED:KNIFE/CUTTING INSTRUMENT' : 'ARMED - KNIFE / CUTTING INSTRUMENT',
    'ARMED: OTHER DANGEROUS WEAPON' : 'ARMED - OTHER DANGEROUS WEAPON',
    'STRONGARM - NO WEAPON' : 'STRONG ARM - NO WEAPON',
    'ATTEMPT: AGGRAVATED' : 'ATTEMPT AGGRAVATED',
    'ATTEMPT: ARMED-KNIFE/CUT INSTR' : 'ATTEMPT ARMED - KNIFE / CUTTING INSTRUMENT',
    'ATTEMPT: ARMED-OTHER DANG WEAP' : 'ATTEMPT ARMED - OTHER DANGEROUS WEAPON',
    'ATTEMPT: STRONGARM-NO WEAPON' : 'ATTEMPT STRONG ARM - NO WEAPON',
    'AGGRAVATED:KNIFE/CUTTING INSTR' : 'AGGRAVATED - KNIFE / CUTTING INSTRUMENT',
    'AGGRAVATED: OTHER DANG WEAPON' : 'AGGRAVATED - OTHER DANGEROUS WEAPON',
    'AGG: HANDS/FIST/FEET NO/MINOR INJURY' : 'AGGRAVATED - HANDS, FISTS, FEET, NO / MINOR INJURY',
    'AGGRAVATED PO: HANDGUN' : 'AGGRAVATED POLICE OFFICER - HANDGUN',
    'AGGRAVATED PO: KNIFE/CUT INSTR' : 'AGGRAVATED POLICE OFFICER - KNIFE / CUTTING INSTRUMENT',
    'AGGRAVATED PO: OTHER DANG WEAP' : 'AGGRAVATED POLICE OFFICER - OTHER DANGEROUS WEAPON',
    'AGG PO HANDS NO/MIN INJURY' : 'AGGRAVATED PO - HANDS, FISTS, FEET, NO / MINOR INJURY',
    'AGG PRO.EMP:KNIFE/CUTTING INST' : 'AGGRAVATED PROTECTED EMPLOYEE - KNIFE / CUTTING INSTRUMENT',
    'AGG PRO.EMP: OTHER DANG WEAPON' : 'AGGRAVATED PROTECTED EMPLOYEE - OTHER DANGEROUS WEAPON',
    'PRO EMP HANDS NO/MIN INJURY' : 'PROTECTED EMPLOYEE - HANDS, FISTS, FEET, NO / MINOR INJURY',
    'AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST' : 'AGGRAVATED DOMESTIC BATTERY - KNIFE / CUTTING INSTRUMENT',
    'AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON' : 'AGGRAVATED DOMESTIC BATTERY - OTHER DANGEROUS WEAPON',
    'AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY' : 'AGGRAVATED DOMESTIC BATTERY - HANDS, FISTS, FEET, SERIOUS INJURY',
    'AGG. DOMESTIC BATTERY - HANDS, FISTS, FEET, SERIOUS INJURY' : 'AGGRAVATED DOMESTIC BATTERY - HANDS, FISTS, FEET, SERIOUS INJURY',
    'AGGRAVATED:KNIFE/CUTTING INSTR' : 'AGGRAVATED - KNIFE / CUTTING INSTRUMENT',
    'AGGRAVATED: OTHER DANG WEAPON' : 'AGGRAVATED - OTHER DANGEROUS WEAPON',
    'PRO EMP HANDS NO/MIN INJURY' : 'PROTECTED EMPLOYEE - HANDS, FISTS, FEET, NO / MINOR INJURY',
    'AGGRAVATED PO: HANDGUN' : 'AGGRAVATED POLICE OFFICER - HANDGUN',
    'AGGRAVATED PO: OTHER FIREARM' : 'AGGRAVATED POLICE OFFICER - OTHER FIREARM',
    'AGGRAVATED PO:KNIFE/CUT INSTR' : 'AGGRAVATED POLICE OFFICER - KNIFE / CUTTING INSTRUMENT',
    'AGGRAVATED PO: OTHER DANG WEAP' : 'AGGRAVATED POLICE OFFICER - OTHER DANGEROUS WEAPON',
    'AGG PO HANDS NO/MIN INJURY' : 'AGGRAVATED POLICE OFFICER - HANDS, FISTS, FEET, NO INJURY',
    'OVER $500' : 'THEFT AMOUNT OVER $300',
    'OVER $300' : 'THEFT AMOUNT OVER $300',
    '$500 AND UNDER' : 'THEFT AMOUNT UNDER $500',
    '$300 AND UNDER' : 'THEFT AMOUNT UNDER $500',
    'FINANCIAL ID THEFT: OVER $300' : 'FINANCIAL IDENTITY THEFT - OVER $300',
    'FINANCIAL IDENTITY THEFT: OVER $300' : 'FINANCIAL IDENTITY THEFT - OVER $300',
    'FINANCIAL ID THEFT:$300 &UNDER' : 'FINANCIAL IDENTITY THEFT - $300 & UNDER',
    'FINANCIAL IDENTITY THEFT: $300 & UNDER' : 'FINANCIAL IDENTITY THEFT - $300 & UNDER',
    'CYCLE, SCOOTER, BIKE W-VIN' : 'CYCLE, SCOOTER, BIKE WITH VIN',
    'THEFT/RECOVERY: AUTOMOBILE' : 'THEFT / RECOVERY - AUTOMOBILE',
    'THEFT/RECOVERY: TRUCK,BUS,MHOME' : 'THEFT / RECOVERY - TRUCK, BUS, MOBILE HOME',
    'THEFT/RECOVERY: CYCLE, SCOOTER, BIKE W-VIN' : 'THEFT / RECOVERY - CYCLE, SCOOTER, BIKE WITH VIN',
    'POS: EXPLOSIVE/INCENDIARY DEV' : 'POSSESSION - EXPLOSIVE / INCENDIARY DEVICE',
    'FINAN EXPLOIT-ELDERLY/DISABLED' : 'FINANCIAL EXPLOITATION OF AN ELDERLY OR DISABLED PERSON',
    'STOLEN PROP: BUY/RECEIVE/POS.' : 'STOLEN PROPERTY - BUY / RECEIVE / POSSESS',
    'THEFT BY LESSEE,MOTOR VEH' : 'THEFT BY LESSEE, MOTOR VEHICLE',
    'THEFT OF LABOR/SERVICES' : 'THEFT OF LABOR / SERVICES',
    'THEFT OF LOST/MISLAID PROP' : 'THEFT OF LOST / MISLAID PROPERTY',
    'TO STATE SUP PROP' : 'TO STATE SUPPORTED PROPERTY',
    'DEFACE IDENT MARKS OF FIREARM' : 'DEFACE IDENTIFICATION MARKS OF FIREARM',
    'POSS FIREARM/AMMO:NO FOID CARD' : 'POSSESS FIREARM / AMMUNITION - NO FOID CARD',
    'SOLICIT FOR BUSINESS' : 'SOLICITING FOR BUSINESS',
    'SALE/DIST OBSCENE MAT TO MINOR' : 'SALE / DISTRIBUTE OBSCENE MATERIAL TO MINOR',
    'AGG CRIMINAL SEXUAL ABUSE' : 'AGGRAVATED CRIMINAL SEXUAL ABUSE',
    'INDECENT SOLICITATION/CHILD' : 'INDECENT SOLICITATION OF A CHILD',
    'INDECENT SOLICITATION/ADULT' : 'INDECENT SOLICITATION OF AN ADULT',
    'SEX RELATION IN FAMILY' : 'SEXUAL RELATIONS IN FAMILY',
    'NON-CONSENSUAL DISSEMINATION PRIVATE SEXUAL IMAGES' : 'NON-CONSENSUAL DISSEMINATION OF PRIVATE SEXUAL IMAGES',
    'ATT AGG CRIMINAL SEXUAL ABUSE' : 'ATTEMPT AGGRAVATED CRIMINAL SEXUAL ABUSE',
    'ENDANGERING LIFE/HEALTH CHILD' : 'ENDANGERING LIFE / HEALTH OF CHILD',
    'CONTRIBUTE DELINQUENCY OF A CHILD' : 'CONTRIBUTE TO THE DELINQUENCY OF CHILD',
    'CRIM SEX ABUSE BY FAM MEMBER' : 'CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER',
    'AGG CRIM SEX ABUSE FAM MEMBER' : 'AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER',
    'SEX ASSLT OF CHILD BY FAM MBR' : 'SEXUAL ASSAULT OF CHILD BY FAMILY MEMBER',
    'AGG SEX ASSLT OF CHILD FAM MBR' : 'AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY MEMBER',
    'CHILD ABDUCTION/STRANGER' : 'CHILD ABDUCTION / STRANGER',
    'POSS: CANNABIS 30GMS OR LESS' : 'POSSESS - CANNABIS 30 GRAMS OR LESS',
    'POSS: CANNABIS MORE THAN 30GMS' : 'POSSESS - CANNABIS MORE THAN 30 GRAMS',
    'MANU/DEL:CANNABIS 10GM OR LESS' : 'MANUFACTURE / DELIVER - CANNABIS 10 GRAMS OR LESS',
    'MANU/DEL:CANNABIS OVER 10 GMS' : 'MANUFACTURE / DELIVER - CANNABIS OVER 10 GRAMS',
    'DELIVER CANNABIS TO PERSON <18' : 'DELIVER CANNABIS TO PERSON UNDER 18',
    'MANU/DELIVER:AMPHETAMINES' : 'MANUFACTURE / DELIVER - AMPHETAMINES',
    'MANU/DELIVER:BARBITUATES' : 'MANUFACTURE / DELIVER - BARBITURATES',
    'MANU/DELIVER:COCAINE' : 'MANUFACTURE / DELIVER - COCAINE',
    'MANU/DELIVER: HEROIN(BRN/TAN)' : 'MANUFACTURE / DELIVER - HEROIN (TAN / BROWN TAR)',
    'MANU/DELIVER: HEROIN (WHITE)' : 'MANUFACTURE / DELIVER -  HEROIN (WHITE)',
    'MANU/DELIVER: HALLUCINOGEN' : 'MANUFACTURE / DELIVER - HALLUCINOGEN',
    'MANU/DELIVER:PCP' : 'MANUFACTURE / DELIVER - PCP',
    'MANU/DELIVER:CRACK' : 'MANUFACTURE / DELIVER - CRACK',
    'MANU/DELIVER:SYNTHETIC DRUGS' : 'MANUFACTURE / DELIVER - SYNTHETIC DRUGS',
    'POSS: AMPHETAMINES' : 'POSSESS - AMPHETAMINES',
    'POSS: BARBITUATES' : 'POSSESS - BARBITURATES',
    'POSS: COCAINE' : 'POSSESS - COCAINE',
    'POSS: HEROIN(BRN/TAN)' : 'POSSESS - HEROIN (TAN / BROWN TAR)',
    'POSS: HEROIN(WHITE)' : 'POSSESS - HEROIN (WHITE)',
    'POSS: HALLUCINOGENS' : 'POSSESS - HALLUCINOGENS',
    'POSS: PCP' : 'POSSESS - PCP',
    'POSS: CRACK' : 'POSSESS - CRACK',
    'POSS: SYNTHETIC DRUGS' : 'POSSESS - SYNTHETIC DRUGS',
    'POSS: HEROIN(BLACK TAR)' : 'POSSESS - HEROIN (BLACK TAR)',
    'MANU/DELIVER:LOOK-ALIKE DRUG' : 'MANUFACTURE / DELIVER - LOOK-ALIKE DRUG',
    'POSS: LOOK-ALIKE DRUGS' : 'POSSESS - LOOK-ALIKE DRUGS',
    'ALTER/FORGE PRESCRIPTION' : 'ALTER / FORGE PRESCRIPTION',
    'SOLICIT NARCOTICS ON PUBLICWAY' : 'SOLICIT NARCOTICS ON PUBLIC WAY',
    'SELL/GIVE/DEL LIQUOR TO MINOR' : 'SELL / GIVE / DELIVER LIQUOR TO MINOR',
    'UNLAWFUL USE/SALE AIR RIFLE' : 'UNLAWFUL USE / SALE OF AIR RIFLE',
    'RESIST/OBSTRUCT/DISARM OFFICER' : '[RESIST / OBSTRUCT / DISARM OFFICER',
    'UNLAWFUL INTERFERE/VISITATION' : 'UNLAWFUL VISITATION INTERFERENCE',
    'VIO BAIL BOND: DOM VIOLENCE' : 'VIOLATION OF BAIL BOND - DOMESTIC VIOLENCE',
    'SEX OFFENDER: FAIL TO REGISTER' : 'SEX OFFENDER - FAIL TO REGISTER',
    'SEX OFFENDER: FAIL REG NEW ADD' : 'SEX OFFENDER - FAIL TO REGISTER NEW ADDRESS',
    'SEX OFFENDER: PROHIBITED ZONE' : 'SEX OFFENDER - PROHIBITED ZONE',
    'OTHER ARSON/EXPLOSIVE INCIDENT' : 'OTHER ARSON / EXPLOSIVE INCIDENT',
    'ATT CRIM SEXUAL ABUSE' : 'ATTEMPT CRIMINAL SEXUAL ABUSE',
    'ARMED: HANDGUN' : 'ARMED - HANDGUN',
    'ARMED: OTHER FIREARM' : 'ARMED - OTHER FIREARM',
    'ATTEMPT: ARMED-HANDGUN' : 'ATTEMPT ARMED - HANDGUN',
    'ATTEMPT: ARMED-OTHER FIREARM' : 'ATTEMPT ARMED - OTHER FIREARM',
    'AGGRAVATED: HANDGUN' : 'AGGRAVATED - HANDGUN',
    'AGGRAVATED: OTHER FIREARM' : 'AGGRAVATED - OTHER FIREARM',
    'AGGRAVATED: HANDGUN' : 'AGGRAVATED - HANDGUN',
    'AGGRAVATED: OTHER FIREARM' : 'AGGRAVATED - OTHER FIREARM',
    'UNLAWFUL USE HANDGUN' : 'UNLAWFUL USE - HANDGUN',
    'UNLAWFUL USE OTHER FIREARM' : 'UNLAWFUL USE - OTHER FIREARM',
    'UNLAWFUL USE OTHER DANG WEAPON' : 'UNLAWFUL USE - OTHER DANGEROUS WEAPON',
    'UNLAWFUL SALE HANDGUN' : 'UNLAWFUL SALE - HANDGUN',
    'UNLAWFUL POSS OF HANDGUN' : 'UNLAWFUL POSSESSION - HANDGUN',
    'UNLAWFUL POSS OTHER FIREARM' : 'UNLAWFUL POSSESSION - OTHER FIREARM',
    'UNLAWFUL POSS AMMUNITION' : 'UNLAWFUL POSSESSION - AMMUNITION',
    'FALSE/STOLEN/ALTERED TRP' : 'FALSE / STOLEN / ALTERED TRP',
    'AGGRAVATED - OTHER' : 'AGGRAVATED - OTHER DANGEROUS WEAPON',
    'AGGRAVATED: OTHER' : 'AGGRAVATED - OTHER',
    'AGGRAVATED - OTHER' : 'AGGRAVATED - OTHER DANGEROUS WEAPON',
    'AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MINOR INJURY' : 'AGGRAVATED POLICE OFFICER - HANDS, FISTS, FEET, NO INJURY',
    'ATT: AUTOMOBILE' : 'ATTEMPT - AUTOMOBILE',
    'ATT: TRUCK, BUS, MOTOR HOME' : 'ATTEMPT - TRUCK, BUS, MOTOR HOME',
    'ATTEMPT: CYCLE, SCOOTER, BIKE W-VIN' : 'ATTEMPT - CYCLE, SCOOTER, BIKE WITH VIN',
    'STOLEN PROPERTY BUY / RECEIVE / POSSESS' : 'STOLEN PROPERTY - BUY / RECEIVE / POSSESS',
    'THEFT BY LESSEE,NON-VEH' : 'THEFT BY LESSEE, NON-MOTOR VEHICLE',
    'RESIST / OBSTRUCT / DISARM OFFICER' : 'RESIST / OBSTRUCT / DISARM OFFICER',
    'AGGRAVATED - OTHER': 'AGGRAVATED - OTHER DANGEROUS WEAPON',
    'AGGRAVATED - OTHER DANGEROUS WEAPON': 'AGGRAVATED - OTHER DANGEROUS WEAPON',
    '[RESIST / OBSTRUCT / DISARM OFFICER' : 'RESIST / OBSTRUCT / DISARM OFFICER',
    '[RESIST / OBSTRUCT / DISARM OFFICER]' : 'RESIST / OBSTRUCT / DISARM OFFICER'
})

# Handled the 'Location Description' column where values have the same meaning but with variations and different spelling
df['Location Description'] = df['Location Description'].replace({
    'RESIDENCE-GARAGE' : 'RESIDENCE - GARAGE',
    'RESIDENCE PORCH/HALLWAY' : 'RESIDENCE - PORCH / HALLWAY',
    'RESIDENTIAL YARD (FRONT/BACK)' : 'RESIDENCE - YARD (FRONT / BACK)',
    'VACANT LOT/LAND' : 'VACANT LOT / LAND',
    'VACANT LOT' : 'VACANT LOT / LAND',
    'HOSPITAL BUILDING/GROUNDS' : 'HOSPITAL BUILDING / GROUNDS',
    'HOTEL/MOTEL' : 'HOTEL / MOTEL',
    'SCHOOL, PUBLIC, BUILDING' : 'SCHOOL - PUBLIC BUILDING',
    'SCHOOL, PUBLIC, GROUNDS' : 'SCHOOL - PUBLIC GROUNDS',
    'SCHOOL, PRIVATE, GROUNDS' : 'SCHOOL - PRIVATE GROUNDS',
    'SCHOOL, PRIVATE, BUILDING' : 'SCHOOL - PRIVATE BUILDING',
    'CHA HALLWAY/STAIRWELL/ELEVATOR' : 'CHA HALLWAY / STAIRWELL / ELEVATOR',
    'CHA PARKING LOT/GROUNDS' : 'CHA PARKING LOT / GROUNDS',
    'GOVERNMENT BUILDING/PROPERTY' : 'GOVERNMENT BUILDING / PROPERTY',
    'CHURCH/SYNAGOGUE/PLACE OF WORSHIP' : 'CHURCH / SYNAGOGUE / PLACE OF WORSHIP',
    'MOVIE HOUSE/THEATER' : 'MOVIE HOUSE / THEATER',
    'FACTORY/MANUFACTURING BUILDING' : 'FACTORY / MANUFACTURING BUILDING',
    'LAKEFRONT/WATERFRONT/RIVERBANK' : 'LAKEFRONT / WATERFRONT / RIVERBANK',
    'NURSING HOME/RETIREMENT HOME' : 'NURSING / RETIREMENT HOME',
    'PARKING LOT/GARAGE(NON.RESID.)' : 'PARKING LOT / GARAGE (NON RESIDENTIAL)',
    'POLICE FACILITY/VEH PARKING LOT' : 'POLICE FACILITY / VEHICLE PARKING LOT',
    'TAVERN/LIQUOR STORE' : 'TAVERN / LIQUOR STORE',
    'SPORTS ARENA/STADIUM' : 'SPORTS ARENA / STADIUM',
    'MEDICAL/DENTAL OFFICE' : 'MEDICAL / DENTAL OFFICE'
})

successfully captialized the Block column


In [14]:
# df['Location Description'] = df['Location Description'].replace({
#     'RESIDENCE-GARAGE' : 'RESIDENCE - GARAGE',
#     'RESIDENCE PORCH/HALLWAY' : 'RESIDENCE - PORCH / HALLWAY',
#     'RESIDENTIAL YARD (FRONT/BACK)' : 'RESIDENCE - YARD (FRONT / BACK)',
#     'VACANT LOT/LAND' : 'VACANT LOT / LAND',
#     'VACANT LOT' : 'VACANT LOT / LAND',
#     'HOSPITAL BUILDING/GROUNDS' : 'HOSPITAL BUILDING / GROUNDS',
#     'HOTEL/MOTEL' : 'HOTEL / MOTEL',
#     'SCHOOL, PUBLIC, BUILDING' : 'SCHOOL - PUBLIC BUILDING',
#     'SCHOOL, PUBLIC, GROUNDS' : 'SCHOOL - PUBLIC GROUNDS',
#     'SCHOOL, PRIVATE, GROUNDS' : 'SCHOOL - PRIVATE GROUNDS',
#     'SCHOOL, PRIVATE, BUILDING' : 'SCHOOL - PRIVATE BUILDING',
#     'CHA HALLWAY/STAIRWELL/ELEVATOR' : 'CHA HALLWAY / STAIRWELL / ELEVATOR',
#     'CHA PARKING LOT/GROUNDS' : 'CHA PARKING LOT / GROUNDS',
#     'GOVERNMENT BUILDING/PROPERTY' : 'GOVERNMENT BUILDING / PROPERTY',
#     'CHURCH/SYNAGOGUE/PLACE OF WORSHIP' : 'CHURCH / SYNAGOGUE / PLACE OF WORSHIP',
#     'MOVIE HOUSE/THEATER' : 'MOVIE HOUSE / THEATER',
#     'FACTORY/MANUFACTURING BUILDING' : 'FACTORY / MANUFACTURING BUILDING',
#     'LAKEFRONT/WATERFRONT/RIVERBANK' : 'LAKEFRONT / WATERFRONT / RIVERBANK',
#     'NURSING HOME/RETIREMENT HOME' : 'NURSING / RETIREMENT HOME',
#     'PARKING LOT/GARAGE(NON.RESID.)' : 'PARKING LOT / GARAGE (NON RESIDENTIAL)',
#     'POLICE FACILITY/VEH PARKING LOT' : 'POLICE FACILITY / VEHICLE PARKING LOT',
# })

In [15]:
# Conclusion from Step 2 -- there is some misspelled values and value with same name with different meaning , handled it using mapping method

##### Step 3 - handle null values

In [16]:
df.isnull().sum()

ID                           0
Case Number                  0
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description      1673
Arrest                       0
Domestic                     0
Beat                         0
District                     0
Ward                    336325
Community Area          334906
FBI Code                     0
X Coordinate              6912
Y Coordinate              6912
Year                         0
Updated On                   0
Latitude                  6912
Longitude                 6912
Location                  6912
dtype: int64

In [17]:
# since there is very less number of null values in 'Location Description', 'X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude' columns , so dropped the column
df.dropna(subset=['Location Description', 'X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude'], inplace=True)

In [20]:
ref = (df.groupby(['Block', 'Location Description', 'District'])[['Ward', 'Community Area']].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index()
    .rename(columns={'Ward': 'Ward_Filled', 'Community Area': 'Community_Area_Filled'}))

df = df.merge(ref, on=['Block', 'Location Description', 'District'], how='left')

df['Ward'] = df['Ward'].fillna(df['Ward_Filled'])
df['Community Area'] = df['Community Area'].fillna(df['Community_Area_Filled'])

df = df.drop(columns=['Ward_Filled', 'Community_Area_Filled'])


In [26]:
df.isnull().sum()

ID                      0
Case Number             0
Date                    0
Block                   0
IUCR                    0
Primary Type            0
Description             0
Location Description    0
Arrest                  0
Domestic                0
Beat                    0
District                0
Ward                    0
Community Area          0
FBI Code                0
X Coordinate            0
Y Coordinate            0
Year                    0
Updated On              0
Latitude                0
Longitude               0
Location                0
dtype: int64

In [25]:
# even after handling null values in "Ward" and "Community Area" columns, there are null values present there, since handling these values by taking mean or median or mode will not be recommended , 
# so fill the null values with "-1" , this -1 indicates unknown in numerical column

df['Ward'] = df['Ward'].fillna(-1)
df['Community Area'] = df['Community Area'].fillna(-1)

In [ ]:
# Conclusion from Step 3 - handled null values in ward and community area , and filled the remaining null values  and dropped the null values in 'Location Description', 'X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude' columns, 
# the reason for dropped this values instead of handled is , these column depends on place where the crime happened , since it is a sensitive data filling this data by taking mean, mode, median or grouping with other values is not correct

##### Step 4 - handle duplicate values

In [24]:
df.duplicated().sum()

np.int64(0)

In [ ]:
# Conclusion from Step 4 - there is no duplicate data in the dataset

##### Step 5 - Feature Engineering

In [ ]:
df['Crime_occured_date'] = df['Date'].dt.date
df['Crime_occured_day'] = df['Date'].dt.day
df['Crime_occured_month'] = df['Date'].dt.month
df['Crime_occured_year'] = df['Date'].dt.year
df['Crime_occured_time'] = df['Date'].dt.time
df['Crime_occured_day_of_week'] = df['Date'].dt.day_name()
df['Crime_occured_is_weekend'] = df['Date'].dt.day_name().isin(['Saturday', 'Sunday'])

def categorize_time(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'
    
df['Crime_occured_time'] = df['Date'].dt.hour.apply(categorize_time)


In [ ]:
df['Record_last_updated_date'] = df['Updated On'].dt.date
df['Record_last_updated_day'] = df['Updated On'].dt.day
df['Record_last_updated_month'] = df['Updated On'].dt.month
df['Record_last_updated_year'] = df['Updated On'].dt.year
df['Record_last_updated_time'] = df['Updated On'].dt.time